# Задание 1

In [8]:
# Импортируем необходимые библиотеки
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Загружаем данные
digits = load_digits()

# Разделяем данные на признаки и целевую переменную
X, y = digits.data, digits.target

# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Проверим размеры
print("Размер обучающей выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)


Размер обучающей выборки: (1257, 64)
Размер тестовой выборки: (540, 64)


# Задание 2

In [9]:
# Импортируем необходимые библиотеки
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time

# Создаем модель дерева решений
base_estimator = DecisionTreeClassifier(random_state=42)

# Создаем модель баггинга
bagging = BaggingClassifier(estimator=base_estimator, n_estimators=100, random_state=42)

# Замеряем время тренировки модели баггинга
start_time = time.time()
bagging.fit(X_train, y_train)
train_time = time.time() - start_time

# Предсказываем на тестовых данных
y_pred_bagging = bagging.predict(X_test)

# Оцениваем точность
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)

# Выводим результаты
print(f"Время обучения баггинг модели: {train_time:.4f} секунд")
print(f"Точность баггинг модели: {accuracy_bagging:.4f}")


Время обучения баггинг модели: 3.9054 секунд
Точность баггинг модели: 0.9519


# Задание 3

In [10]:
# Импортируем библиотеку для бустинга
from sklearn.ensemble import GradientBoostingClassifier

# Создаем модель бустинга
boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Замеряем время тренировки модели бустинга
start_time = time.time()
boosting.fit(X_train, y_train)
train_time = time.time() - start_time

# Предсказываем на тестовых данных
y_pred_boosting = boosting.predict(X_test)

# Оцениваем точность
accuracy_boosting = accuracy_score(y_test, y_pred_boosting)

# Выводим результаты
print(f"Время обучения бустинг модели: {train_time:.4f} секунд")
print(f"Точность бустинг модели: {accuracy_boosting:.4f}")


Время обучения бустинг модели: 6.8317 секунд
Точность бустинг модели: 0.9685


# Задание 4

In [11]:
print("\nСравнение моделей:")
print(f"Точность баггинг модели: {accuracy_bagging:.4f}")
print(f"Точность бустинг модели: {accuracy_boosting:.4f}")
print(f"Время обучения баггинг модели: {train_time:.4f} секунд")
print(f"Время обучения бустинг модели: {train_time:.4f} секунд")



Сравнение моделей:
Точность баггинг модели: 0.9519
Точность бустинг модели: 0.9685
Время обучения баггинг модели: 6.8317 секунд
Время обучения бустинг модели: 6.8317 секунд
